# データストアの操作

データ サイエンティストがローカル ファイル システム上のデータを操作することはかなり一般的ですが、エンタープライズ環境では、複数のデータ サイエンティストがアクセスできる中央の場所にデータを格納する方が効果的です。このラボでは、クラウドにデータを格納し、Azure Machine Learning *データストア*を使用してデータにアクセスします。

> **重要**: この Notebooks のコードでは、 ラボ 4Aの最初の 2 つのタスクを完了していることを前提としています。完了していない場合は、今すぐ完了してください。


## ワークスペースに接続する

Azure の機械学習 SDK を使用してデータストアにアクセスするには、ワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## ワークスペースでデータストアを表示する

ワークスペースには、[前のタスク](labdocs/Lab04A.md)で作成した **aml_data** データストアを含む、いくつかのデータストアが含まれています。

次のコードを実行して、*既定の*データストアを取得し、すべてのデータストアを一覧表示し、どれが既定であるかを示します。

In [ ]:
from azureml.core import Datastore

# 既定のデータストアを取得する
default_ds = ws.get_default_datastore()

# すべてのデータストアを列挙し、どちらが既定かを示す
for ds_name in ws.datastores:
    print(ds_name, "- Default =", ds_name == default_ds.name)

## 操作するデータストアを取得する

**aml_data** データストアを操作する必要があるため、名前で取得する必要があります。

In [ ]:
aml_datastore = Datastore.get(ws, 'aml_data')
print(aml_datastore.name,":", aml_datastore.datastore_type + " (" + aml_datastore.account_name + ")")

## 既定のデータストアを設定する

このコースでは主に **aml_data** データストアを操作しますので、便利なように、既定のデータストアに設定してください。

In [ ]:
ws.set_default_datastore('aml_data')
default_ds = ws.get_default_datastore()
print(default_ds.name)

## データをデータストアにアップロードする

これで、操作するデータストアを特定したので、実際に実験スクリプトが実行されている場所に関係なく、ワークスペースで実行されている実験にアクセスできるように、ローカル ファイル システムからファイルをアップロードできます。

In [ ]:
default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # 糖尿病 CSV ファイルを /data にアップロードする
                       target_path='diabetes-data/', # データストアのフォルダー パスに入れる
                       overwrite=True, # 同じ名前の既存のファイルを置き換える
                       show_progress=True)

## データストアからモデルをトレーニングする

上記のコード セルにファイルをアップロードすると、コードから*データ参照*が返されることに注意してください。データ参照は、スクリプトが実行されている場所に関係なく、データストア内のフォルダーへのパスをスクリプトに渡す方法を提供し、スクリプトがデータストアの場所にあるデータにアクセスできるようにします。

次のコードは、糖尿病 CSV ファイルをアップロードした **diabetes-data** フォルダーへの参照を取得し、*ダウンロード*用のデータ参照を具体的に構成します。つまり、フォルダーのコンテンツを、データ参照が使用されているコンピューティング コンテキストにダウンロードするために使用できます。データのダウンロードは、ローカル コンピューティングで処理される少量のデータに適しています。リモート コンピューティングで操作する場合は、データストアの場所を*マウント*して、データソースから直接データを読み取るようにデータ参照を構成することもできます。

> **詳細情報**: データストアの使用の詳細については、[Azure ML のドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-access-data)を参照してください。

In [ ]:
data_ref = default_ds.path('diabetes-data').as_download(path_on_compute='diabetes_data')
print(data_ref)

トレーニング スクリプトでデータ参照を使用するには、そのパラメーターを定義する必要があります。次の 2 つのコード セルを実行して作成します。

1.**diabetes_training_from_datastore** という名前のフォルダー
2.渡されたデータ参照パラメーターによって参照されるフォルダー内のすべての CSV ファイルのトレーニング データを使用して分類モデルをトレーニングするスクリプト。

In [ ]:
import os

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_training_from_datastore'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created.')

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import os
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# パラメーターを取得する
parser = argparse.ArgumentParser()
parser.add_argument('--regularization', type=float, dest='reg_rate', default=0.01, help='regularization rate')
parser.add_argument('--data-folder', type=str, dest='data_folder', help='data folder reference')
args = parser.parse_args()
reg = args.reg_rate

# 実験実行コンテキストを取得する
run = Run.get_context()

# データ参照から糖尿病データを読み込む
data_folder = args.data_folder
print("Loading data from", data_folder)
# すべてのファイルを読み込み、その内容を単一のデータフレームとして連結する
all_files = os.listdir(data_folder)
diabetes = pd.concat((pd.read_csv(os.path.join(data_folder,csv_file)) for csv_file in all_files))

# フィーチャーとラベルを分離する
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# ロジスティック回帰モデルをトレーニングする
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

スクリプトは、渡されたデータ参照からトレーニング データをパラメーターとして読み込むため、実験の実行時にファイル参照を渡すスクリプト パラメーターを設定する必要があるだけです。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment, Environment
from azureml.widgets import RunDetails

# Python 環境を作成する
env = Environment("env")
env.python.user_managed_dependencies = True
env.docker.enabled = False

# パラメーターをセットアップする
script_params = {
    '--regularization': 0.1, # 正規化率
    '--data-folder': data_ref # データストアからファイルをダウンロードするためのデータ参照
}


# Estimator を作成する
estimator = Estimator(source_directory=experiment_folder,
                      entry_script='diabetes_training.py',
                      script_params=script_params,
                      compute_target = 'local',
                      environment_definition=env
                   )

# 実験を作成する
experiment_name = 'diabetes-training'
experiment = Experiment(workspace=ws, name=experiment_name)

# 実験を実行する
run = experiment.submit(config=estimator)
# 実行中に実行の詳細を表示する
RunDetails(run).show()
run.wait_for_completion()

初めて実験を実行すると、Python 環境のセットアップに時間がかかる場合があります。以降の実行はより高速になります。

実験が完了したら、ウィジェットで出力ログを表示し、データ ファイルがダウンロードされたことを確認します。

すべての実験と同様に、[Azure ML Studio](https://ml.azure.com) で実行された実験の詳細を表示したり、生成されたメトリックとファイルを取得するコードを書き込んだりできます。

In [ ]:
# 指標の記録を取得する
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

もう一度、実験によってトレーニングされたモデルを登録できます。

In [ ]:
from azureml.core import Model

# モデルを登録する
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Using Datastore'}, properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 登録済みモデルを一覧表示する
print("Registered Models:")
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

この演習では、*データストア*の形式でデータを操作するためのいくつかのオプションについて説明しました。

Azure Machine Learning では、データ抽象化をさらに高いレベルである*データセット形式*で提供しています。これについては、次に説明します。